In [1]:
import warnings
warnings.simplefilter('ignore')
from sklearn.tree import DecisionTreeClassifier
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.externals import joblib



class PandasDummies(BaseEstimator, TransformerMixin):
    def transform(self, X, *_):
        return pd.get_dummies(X)
    
    def fit(self, *_):
        return self

In [2]:
teamData=pd.read_csv("../team/cleaned_team_stats.csv")
teamData.columns



Index(['team_id', 'variable', 'first_downs', 'first_downs_by_penalty',
       'third_down_percentage', 'fourth_down_percentage',
       'average_interception_yards', 'average_kickoff_return_yards',
       'average_punt_return_yards', 'interceptions', 'net_average_punt_yards',
       'net_passing_yards', 'net_passing_yards_per_game',
       'passing_first_downs', 'passing_touchdowns', 'rushing_first_downs',
       'rushing_attempts', 'rushing_touchdowns', 'rushing_yards',
       'rushing_yards_per_game', 'total_offensive_plays', 'total_points',
       'total_points_per_game', 'total_touchdowns', 'total_offensive_yards',
       'yards_per_game', 'yards_per_pass_attempt', 'yards_per_rush_attempt',
       'completed_passes', 'attempted_passes', 'field_goals_completed',
       'field_goals_attempted', 'total_fumbles', 'defensive_interception',
       'yards_after_interception', 'total_kickoffs_received',
       'yards_off_kickoff_received', 'total_punts_received',
       'yards_off_punts_re

In [41]:
X = teamData[[  'third_down_percentage',
 'yards_per_pass_attempt',
 'passing_touchdowns',
  'rushing_touchdowns','rushing_yards_per_game', 'net_passing_yards_per_game'
 
 
       ]]

y= teamData['total_points'].values.reshape(-1, 1)

In [42]:
def linerregressionRun(X, y, times):
    
    MSEresults=[]
    R2results=[]
    rffit=[]
    rfscore=[]
    i=0
    rfFeatures=[]
    steps=[]
   
    while i<times:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)
        model = make_pipeline(PandasDummies(), StandardScaler(), LinearRegression())
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        MSE = mean_squared_error(y_test, predictions)
        r2 = model.score(X_test, y_test)
        print(f"MSE: {MSE}, R2: {r2}")
        MSEresults.append(MSE)
        R2results.append(r2)
        joblib.dump(model, F"{i}my_model.pkl")
        rf = RandomForestRegressor(n_estimators=200, max_leaf_nodes=5)
        rf = rf.fit(X_train, y_train)
        acc = rf.score(X_test, y_test)
        rfF=sorted(zip(rf.feature_importances_, X.columns), reverse=True)
        rfFeatures.append(rfF)
        steps.append(model.steps)
        rffit.append(rf)
        
        
        rfscore.append(acc)
        print(f"accuracy = {acc}")
        i=i+1
    results={"MSE":MSEresults, 
             "r2":R2results,
             "RandomForest":rffit,
             "RFScore":rfscore,
             "features":rfFeatures,
             "steps":steps
             }
    return pd.DataFrame(results)
    


In [43]:
TUNNING = linerregressionRun(X, y, 5)

MSE: 495.93508742490945, R2: 0.9535129021716252
accuracy = 0.8002315508564704
MSE: 495.93508742490945, R2: 0.9535129021716252
accuracy = 0.7902491141948056
MSE: 495.93508742490945, R2: 0.9535129021716252
accuracy = 0.7731309354464032
MSE: 495.93508742490945, R2: 0.9535129021716252
accuracy = 0.783515810399236
MSE: 495.93508742490945, R2: 0.9535129021716252
accuracy = 0.7932849772652395


In [44]:
TUNNING.columns

Index(['MSE', 'r2', 'RandomForest', 'RFScore', 'features', 'steps'], dtype='object')

In [45]:
TUNNING["features"][0]

[(0.3538979769839156, 'third_down_percentage'),
 (0.24976970574629698, 'passing_touchdowns'),
 (0.20347851973978132, 'yards_per_pass_attempt'),
 (0.17132097540801386, 'rushing_touchdowns'),
 (0.017607913162239353, 'net_passing_yards_per_game'),
 (0.003924908959752904, 'rushing_yards_per_game')]

In [46]:
TUNNING["RandomForest"][0].base_estimator_


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [48]:
X2 = teamData[[ 'third_down_percentage',
 'yards_per_pass_attempt',
 'passing_touchdowns',
  'rushing_touchdowns',
'rushing_yards_per_game', 'net_passing_yards_per_game',
 ]]

y2= teamData['total_points'].values.reshape(-1, 1)
              

In [49]:


some_totally_random_model = joblib.load("0my_model.pkl")

In [50]:
X2

,third_down_percentage,yards_per_pass_attempt,passing_touchdowns,rushing_touchdowns,rushing_yards_per_game,net_passing_yards_per_game
0,42.99,6.8,19,30,208.0,182.3
1,39.33,7.7,22,12,141.3,288.8
2,34.84,7.8,19,20,188.2,203.3
3,45.88,9.5,33,32,186.7,298.4
4,46.20,11.9,10,52,360.5,95.2
5,43.87,8.0,35,35,180.8,309.0
6,38.98,6.2,18,15,161.3,169.6
7,37.81,7.0,26,16,137.2,254.8
8,38.46,8.0,19,33,243.2,206.1
9,42.51,7.6,20,18,138.3,281.3


In [51]:

X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.40, random_state=42)

In [52]:
results=some_totally_random_model.predict(X2)

In [53]:
results


array([[400.36620743],
       [313.04559548],
       [333.35912647],
       [527.26882044],
       [506.01754862],
       [553.94246082],
       [285.34728017],
       [354.15572321],
       [427.09762384],
       [340.04761023],
       [559.61104387],
       [244.75432053],
       [393.29360436],
       [638.64164097],
       [285.43559878],
       [396.8347856 ],
       [225.86879717],
       [455.44200855],
       [348.9892348 ],
       [258.97459473],
       [433.81037584],
       [265.83348995],
       [335.23288468],
       [420.02002338],
       [381.88833026],
       [418.54450606],
       [473.13861886],
       [424.3382568 ],
       [308.04259362],
       [349.99697979],
       [597.51606154],
       [415.37000875],
       [335.3493783 ],
       [477.85334964],
       [381.56343091],
       [253.37246121],
       [315.87998431],
       [411.37984998],
       [297.39704245],
       [319.66815489],
       [268.41453975],
       [416.86593551],
       [465.80089256],
       [343

In [54]:
teamData['total_points'][60]

324

In [56]:
def mlyards(one, two, three, four, five, six):
    #create df with these 5 variables with column names from model
    data={'third_down_percentage': one,
        'yards_per_pass_attempt':two,
        'passing_touchdowns':three,
        'rushing_touchdowns':four,
        'rushing_yards_per_game':five,
         'net_passing_yards_per_game':six}
    df=pd.DataFrame(data, index=[0])
    #import model
    some_totally_random_model = joblib.load("0my_model.pkl")
    #run number with model
    results=some_totally_random_model.predict(df)
    #return result as int
    return results

In [59]:
mlyards(50,4.53,2,205,77)

array([[102.71043543]])